In [6]:
%%capture

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
import os

matplotlib.rcParams.update({'font.size': 14})
plt.tight_layout()

_TODO: add documentation info from cohort maps._

# Data Exploration

Let's take a look at Columbia's average net prices since 2008 (when colleges began reporting student aid data).

Note that the years in the scorecard data filenames correspond to when the data was collected. For example, the 2016-17 data file describes data collected in 2016-17, so it actually describes the 2015-16 academic year.

In [7]:
data = {}
scorecards = {}

# Read every scorecard file from 2009 to 2016 into a list.
for year in range(9, 17):
    fname = f'MERGED20{year:02}_{(year + 1):02}_PP.csv'
    df = pd.read_csv('../data/scorecard/' + fname, low_memory=False)
    data[2000 + year - 1] = df
    # Isolate Columbia data
    scorecards = df.set_index('INSTNM').loc['Columbia University in the City of New York']
    print('Finished reading', fname)

Finished reading MERGED2009_10_PP.csv
Finished reading MERGED2010_11_PP.csv
Finished reading MERGED2011_12_PP.csv
Finished reading MERGED2012_13_PP.csv
Finished reading MERGED2013_14_PP.csv
Finished reading MERGED2014_15_PP.csv
Finished reading MERGED2015_16_PP.csv
Finished reading MERGED2016_17_PP.csv


The College Scorecard only has data up to academic year (AY) 2015-16.

IPEDS has [more recent financial aid data (in the "Net Price" section)](https://nces.ed.gov/ipeds/datacenter/institutionprofile.aspx?unitId=acababb2abb1) that we have to hard code into the scorecard list.

To get the average annual cost of attendance, go to Columbia's [2016 Institutional Characteristics Survey](https://nces.ed.gov/ipeds/datacenter/facsimile.aspx?action=pdf&unitid=acb4abacb0ab&year=2016&surveyNumber=1) in the IPEDS Data Center. On pages 11-12 is the makeup of the cost of attendance for 2016. To calculate the cost of attendance, add "Tuition + fees total", "Books and supplies", and "On-campus Room and board and other expenses". 

Though the College Scorecard does not explain how it factors in on-campus and off-campus costs of attendance, it seems to only use on-campus costs for its data. This is probably because the average annual cost is defined for first-time, full-time undergraduates, and Columbia requires these students to live on campus.

In [8]:
scorecards[2016] = {
    'NPT41_PRIV': 10917.0,
    'NPT42_PRIV': 6596.0,
    'NPT43_PRIV': 7648.0,
    'NPT44_PRIV': 17463.0,
    'NPT45_PRIV': 42912.0,
    'COSTT4_A': 71785,
}

IndexError: index 2016 is out of bounds for axis 0 with size 1898

To adjust prices for inflation, we will use consumer price index data to make price data equivalent to 2016 dollars. The month of 2016 we adjust to depends on when each category of data was calculated. For example, net price data is reported around December, whereas debt data is reported around October.

In [ ]:
cpiData = pd.read_csv('../data/ConsumerPriceIndexData.csv').set_index('Year')

# Returns a function that adjusts nominal dollars to [month] 2016 dollars.
def inflationAdjuster(month):
    finalCpi = cpiData.loc[2016][month]
    def adjust(dollars, year):
        if dollars is None:
            return
        initCpi = cpiData.loc[year][month]
        return dollars * finalCpi / initCpi
    return adjust

In [ ]:
colNames = {
    'NPT41_PRIV': 'ANP for 0-30,000 fam income',
    'NPT42_PRIV': 'ANP for 30,001-48,000 fam income',
    'NPT43_PRIV': 'ANP for 48,001-75,000 fam income',
    'NPT44_PRIV': 'ANP for 75,001-110,000 fam income',
    'NPT45_PRIV': 'ANP for 110,000+ fam income',
    'COSTT4_A': 'Total cost of attendance',
}

YEARS = [i for i in range(2008, 2017)]

def getPrices(col):
    # Net prices are calculated around Nov-Dec.
    # IPEDS help desk when asked if I could use December CPI's to adjust
    # net prices for inflation: "Probably." —Mike (the one person who
    # runs the help desk). 🧐
    adjust = inflationAdjuster('Dec')
    return [adjust(scorecards[year][col], year) for year in YEARS]

def comparePriceLines(*cols, title=None):
    plt.figure(figsize=(10,10))
    for col in cols:
        plt.plot(YEARS, getPrices(col), label=colNames.get(col, col))        
    plt.legend()
    plt.xlabel('Academic year')
    plt.ylabel('Average net price in December 2016 dollars')
    plt.ylim(bottom=0)
    if title:
        plt.title(title)

In [ ]:
comparePriceLines(
    'NPT41_PRIV',
    'NPT42_PRIV',
    'NPT43_PRIV',
    title='Average net price (ANP) for lowest 3 income quintiles'
)

In my initial analysis, I made the mistake of making a hasty conclusion about Columbia's increasingly inequitable financial aid distribution. However, when we graph ANP's for all income quintiles with the total cost of attendance...

In [ ]:
comparePriceLines(
    'NPT41_PRIV',
    'NPT42_PRIV',
    'NPT43_PRIV',
    'NPT44_PRIV',
    'NPT45_PRIV',
    'COSTT4_A',
    title='Average net price (ANP) for every income quintile compared to total cost'
)

...we see that the differences between the ANP's of the three lowest income quintiles are relatively small compared to the differences in total cost. Thus, no immediate conclusion about financial aid can be made. This data only tells us further reporting is necessary to explain the disparities.

Average net price is defined as the total cost of attendance minus the average aid awarded to students of that income quintile. Let's graph the average aid awarded.

In [ ]:
colNames = {
    'NPT41_PRIV': 'Average aid to 0-30,000 fam income',
    'NPT42_PRIV': 'Average aid to 30,001-48,000 fam income',
    'NPT43_PRIV': 'Average aid to 48,001-75,000 fam income',
    'NPT44_PRIV': 'Average aid to 75,001-110,000 fam income',
    'NPT45_PRIV': 'Average aid to 110,000+ fam income',
    'COSTT4_A': 'Total cost of attendance',
}

def getAid(col):
    adjust = inflationAdjuster('Dec')
    if col == 'COSTT4_A':
        return getPrices(col)
    return [
        adjust(scorecards[year]['COSTT4_A'] - scorecards[year][col], year) for year in YEARS
    ]

def compareAidLines(*cols, title=None):
    plt.figure(figsize=(10,10))
    for col in cols:
        plt.plot(YEARS, getAid(col), label=colNames.get(col, col))
    plt.legend()
    plt.xlabel('Academic year')
    plt.ylabel('Average net price in December 2016 dollars')
    plt.ylim(bottom=0)
    if title:
        plt.title(title)

In [ ]:
compareAidLines(
    'NPT41_PRIV',
    'NPT42_PRIV',
    'NPT43_PRIV',
    'NPT44_PRIV',
    'NPT45_PRIV',
    'COSTT4_A',
    title='Average aid awarded for every income quintile in the last decade'
)

We see now more clearly now that financial aid for the lower four income quintiles are growing at similar rates

In [ ]:
# Retrieves percent increase column data since 2009 for every year
def getPriceIncreases(col):
    data = getPrices(col)
    initial = data[0]
    return [
        (d - initial) / initial
        for d in data
    ]

def compareAggregatePriceIncreases(*cols, title=None):
    plt.figure(figsize=(10,10))
    for col in cols:
        plt.plot(YEARS, getPriceIncreases(col), label=colNames.get(col, col))
    plt.legend()
    plt.xlabel('Academic year')
    plt.ylabel('Change since 2009')
    plt.axhline(y=0, linestyle='--', color='#aaaaaa')
    if title:
        plt.title(title)    

In [ ]:
compareAggregatePriceIncreases(
    'NPT41_PRIV',
    'NPT42_PRIV',
    'NPT43_PRIV',
    'NPT44_PRIV',
    'NPT45_PRIV',
    title="ANP change since 2009 for each income quintile."
)

# Debt

According to the [College Scorecard Data Documentation (page 18)](https://collegescorecard.ed.gov/assets/FullDataDocumentation.pdf), debt data for each cohort (e.g. low income, medium income, pell grant students) is produced for each fiscal year. Fiscal years end on September 30, so the CPI's I use for each year are from September.

In [ ]:
colNames = {
    'LO_INC_DEBT_MDN': 'median debt for 0-30k students',
    'MD_INC_DEBT_MDN': 'median debt for 30k-75k students',
    'HI_INC_DEBT_MDN': 'median debt for 75k+ students',
}

def getDebtData(col):
    adjust = inflationAdjuster('Sep')
    # No debt data is available for AY 2016-17.
    return [
        adjust(float(scorecards[year][col]), year) for year in YEARS[:-1]
    ] + [None]

def compareDebtLines(*cols, title=None):
    plt.figure(figsize=(10,10))
    for col in cols:
        plt.plot(YEARS, getDebtData(col), label=colNames.get(col, col))        
    plt.legend()
    plt.xlabel('Academic year')
    plt.ylabel('Median debt (in nominal dollars)')
    plt.ylim(bottom=0)
    if title:
        plt.title(title)

In [ ]:
compareDebtLines('LO_INC_DEBT_MDN', 'MD_INC_DEBT_MDN', 'HI_INC_DEBT_MDN')

# Context

How does the inequitable net price growths at Columbia compare with other elite institutions?

In [ ]:
def getData(uni, col):
    adjust = inflationAdjuster('Dec')
    return [
        adjust(float(data[year].loc[data[year]['INSTNM'] == uni][col]), year)
        for year in YEARS[:-1]
    ]

class ComparisonGraph:
    def __init__(self, unis, cols):
        self.unis = unis
        self.cols = cols
        self.width = 3
        self.height = math.ceil(len(unis) / self.width)

    def graph(self):
        plt.figure(figsize=(10,20))
        for i in range(len(self.unis)):
            self.graphUni(i)
            
    def graphUni(self, index):
        plt.subplot(self.width, self.height, index + 1)
        uni = self.unis[index]
        for col in self.cols:
            plt.plot(YEARS[:-1], getData(uni, col), label=col)
        plt.legend()
        plt.xlabel('Academic year')
        plt.ylabel('Average net price in December 2016 dollars')
        plt.ylim(bottom=0)
        plt.title(uni)
                
colNames = {
    'NPT41_PRIV': 'ANP for 0-30,000 fam income',
    'NPT42_PRIV': 'ANP for 30,001-48,000 fam income',
    'NPT43_PRIV': 'ANP for 48,001-75,000 fam income',
    'NPT44_PRIV': 'ANP for 75,001-110,000 fam income',
    'NPT45_PRIV': 'ANP for 110,000+ fam income',
    'COSTT4_A': 'Total cost of attendance',
}

In [ ]:
unis = ['Columbia University in the City of New York', 'Stanford University', 'Brown University']
cols = ['NPT41_PRIV', 'NPT42_PRIV', 'NPT43_PRIV']

c = ComparisonGraph(unis, cols)
c.graph()